# Détection d'arêtes

In [ ]:
# settings for display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# turn off matplotlib figure axes, else we need to call axes('off') everytime
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
plt.set_cmap('gray') 
# mpl.rc('axes.spines',top=False,bottom=False,left=False,right=False)
# mpl.rc(('xtick','ytick'),color=(1,1,1,0))
mpl.rc('font', size=20)

import warnings
warnings.filterwarnings("ignore")

# import necessarry libs
from imageio import imread
import numpy as np
from scipy.signal import convolve2d as conv2
from skimage.color import rgb2gray
from skimage.transform import resize
from scipy.fftpack import fft2, fftshift, ifft2

# define gaussian filter
def f_gauss(size, sigma):
    # gaussian filter
    x, y = np.mgrid[-size[0]//2 + 1:size[0]//2 + 1, -size[1]//2 + 1:size[1]//2 + 1]
    g = np.exp(-((x**2 + y**2)/(2.0*sigma**2)))
    return g/g.sum()

# define average filter
def f_average(size):
    k = np.ones(size)
    return k/k.sum()

def f_log_kernel(size, std):
    x, y = np.meshgrid(np.linspace(-size, size, 2*size+1), np.linspace(-size, size, 2*size+1))
    arg = -(x**2 + y**2) / (2*std**2)
    h = np.exp(arg)
    h[h < 1e-10 * h.max()] = 0
    h = h/h.sum() if h.sum() != 0 else h
    h1 = h*(x**2 + y**2 - 2*std**2) / (std**4)
    return h1 - h1.mean()

In [ ]:
im = rgb2gray(imread('5. Filtrage spatial/img.jpg')/255.0)

In [ ]:
# filtre gaussien
g = f_gauss([15,15], 2);

# plot the images and filtrée
%matplotlib notebook
fig = plt.figure(figsize=(10,5), num='Filtre gaussien')

ax1 = fig.add_subplot(1, 2, 1)
ax1.imshow(g); ax1.axis('off'); ax1.set_title('Image gaussien 2D')
ax2 = fig.add_subplot(1, 2, 2, projection='3d')
X = np.linspace(-1, 1, 15)
Y = np.linspace(-1, 1, 15)
X, Y = np.meshgrid(X, Y)
fmap=ax2.plot_surface(X,Y,g); ax2.set_title('Image gaussien 3D');
plt.tight_layout()

In [ ]:
# image filtrée
imG = conv2(im, g, mode='same');

# dérivée de l'image
imD = conv2(im, np.asarray([-1, 1])[:,np.newaxis],'same');
# dérivée de l'image floue
imDG = conv2(imG, np.asarray([-1, 1])[:,np.newaxis], 'same');

# plot
fig = plt.figure(figsize=(10,5), num='Filtre')

ax1 = fig.add_subplot(1, 2, 1)
ax1.imshow(np.log(np.abs(imD))); ax1.axis('off'); ax1.set_title('Dérivée de l''image')
ax2 = fig.add_subplot(1, 2, 2)
ax2.imshow(np.log(np.abs(imDG))); ax2.axis('off'); ax2.set_title('Dérivée de l''image floue');
plt.tight_layout()

In [ ]:
# dérivée de gaussienne
dg = conv2(g,np.asarray([-1, 1])[:,np.newaxis],'same');

# Laplacien d'une gaussienne
lg = f_log_kernel(15, 2);

imDG = conv2(im,dg,'same');
imLOG = conv2(im,lg,'same');

# plot
fig = plt.figure(figsize=(10,5), num='Gaussienne & LOG')
# dérivée (en X)
ax1 = fig.add_subplot(1, 2, 1)
ax1.imshow(imDG); ax1.axis('off'); ax1.set_title('Dérivée Gaussienne')
# LOG
ax2 = fig.add_subplot(1, 2, 2, sharex = ax1, sharey = ax1)
ax2.imshow(imLOG); ax2.axis('off'); ax2.set_title('LOG');
plt.tight_layout()